# Data Cleaning and Preparation

---

During the course of doing data analysis and modeling, a significant amount of time is spent on data preparation: loading, cleaning, transforming, and rearranging. Such tasks often takes 80% or more of an analyst’s time. Sometimes the way that data is stored in files or databases is not in the right format for a particular task. Fortunately, pandas, along with the built-in Python language features, provides you with a high-level, flexible, and fast set of tools to enable you to manipulate data into the right form.


### Lecture outline

---

* Finding and Filling Missing Values


* Removing Duplicate Values


* Replacing Values


* Discretization and Binning


* Detecting Outliers


* String Manipulations


* Variable Transformation

## Finding and Filling Missing Values

---

Missing values are pretty common in data cleaning activities. And, they can be there for any number of reasons.


For instance, if you are running a survey and a respondent didn't answer a question the missing value is
actually an omission. This kind of missing data is called **Missing at Random** if there are other variables
that might be used to predict the variable which is missing. If there is no relationship to other variables, then  we call this data **Missing Completely at Random (MCAR)**, in other words, missing is independent of the observed and unobserved data. **Missing not at random (MNAR)**. When data are MNAR, the fact that the data are missing is systematically related to the unobserved data, that is, the missingness is related to events or factors which are not measured by the researcher.


Pandas works with missing data as painless as possible. For example, all of the descriptive statistics on pandas objects exclude missing data by default.


> Not only `NaN`, `NA`, `N/A`, `NAT`, `NULL`, and `None` are missing values. There can be other missing values in the data. That's why we need always to know our data

#### Reference

[Types of Missing Data](https://www.ncbi.nlm.nih.gov/books/NBK493614/)


[Working with missing data](https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html)

![alt text](images/missing_data.jpeg "Title")

In [ ]:
import pandas as pd

import numpy as np

In [ ]:
weather = pd.read_csv("data/weather.csv")

weather.head()

### Detecting Missing Values

In [ ]:
weather.isnull() # Retruns boolean seris. True denotes missing value


weather.isna() # Same as "isnull()" method

In [ ]:
weather.isnull().any() # Shows all columns with missing value

In [ ]:
weather[weather.isnull().any(axis=1)] # Show all rows with missing values

# Something weird happens in "MIN_TEMP_GROUND" column - We'll see later

In [ ]:
weather.isnull().all(axis=1).any() # Are there any rows with only null values?

In [ ]:
weather.notnull().all() # Are there any columns with no null values at all?

In [ ]:
weather["MIN_TEMP_GROUND"].head(30) # Do you see pattern?

In [ ]:
every_6th_row_index = pd.Series(range(5, len(weather), 6)) # Indices for every 6th row

every_6th_row_index

In [ ]:
weather["MIN_TEMP_GROUND"][every_6th_row_index].notnull().all() # Are all these rows NOT null?

In [ ]:
weather['MIN_TEMP_GROUND'].drop(every_6th_row_index).isnull().all() # Are all other rows null?

### Handling Missing Values

---

The strategy of handling missing values depends on the type of missing value and/or the problem and data at hand. We may have huge amount of data and dropping missing values will not affect our aims, or we may have small amount of data and it's desirable to impute the missing values.

Let see how can we drop/remove missing values in rows and columns.

In [ ]:
series = pd.Series([1, np.nan, 3.5, np.nan, 7, 10, np.nan])

series

In [ ]:
series.dropna() # Removes missing values


series[series.notnull()] # Same as above

Dropping missing values from DataFrame is somewhat different from dropping missing values from Series. For that reason, I create sample DataFrame, to show the effect of `dropna()` method on DataFrame.

In [ ]:
missing_df = pd.DataFrame([[1., 6.5, 3.], [1., np.nan, np.nan],
                           [np.nan, np.nan, np.nan], [np.nan, 6.5, 3.]], columns=["a", "b", "c"])

missing_df

In [ ]:
missing_df.dropna(axis=0, how="any") # The shortest solution - drop everything (Generally not good idea!!!)


missing_df.dropna(axis=0, how="all") # Remove rows if all values are missing

In [ ]:
missing_df.dropna(axis=1, how="any") # Drop column if contains at least one missing value

missing_df.dropna(axis=1, how="all") # Drop column if all values are missing

In [ ]:
missing_df.dropna(axis=0, how="any", subset=["a"]) # Filter out missing values by column

Rather than filtering out missing data (and potentially discarding other data along with it), you may want to fill in the “holes” in any number of ways. For most purposes, the `fillna()` method is the workhorse function to use.

In [ ]:
missing_df

In [ ]:
missing_df.fillna(value=999) # Fill missing values with a constant

missing_df.fillna(missing_df.mean()) # Fill missing values with a mean

missing_df.fillna(missing_df.mode().iloc[0]) # Fill missing values with a mode

missing_df.fillna({"a": 999, "b": -1, "c": 100}) # Fill missing values by different fill value for each column

In [ ]:
missing_df

In [ ]:
missing_df.fillna(method="ffill") # Forward Fill - use last valid observation for filling

missing_df.fillna(method="backfill") # Forward fill - use next valid observation to fill gap

## Removing Duplicate Values

---

We may have duplicate values in our data due to several reasons and they can cause some difficulties during data analysis procedure. We have to identify them and then handle them properly. In other words, we have to find unique identifier for each row.

In [ ]:
athletes = pd.read_csv("data/athletes.csv")

athletes.head()

In [ ]:
athletes.duplicated() # Boolean series indicating duplicated rows. Uses all columns to find duplicates

athletes.duplicated(subset=["id", "sex"]) # Uses only two columns for duplicate identification

In [ ]:
athletes[athletes.duplicated()] # Shows which rows are duplicated

In [ ]:
athletes.drop_duplicates() # Removes all duplicate rows


athletes.drop_duplicates(subset=["id", "nationality"]) # Remove duplicates only considering some columns

## Replacing Values

---

There are situations when we just need to replace values in a Pandas Series or DataFrame. For that reason, we can use `replace()` method

In [ ]:
missing_df.replace(to_replace=np.nan, value=999) # Replace all NaN's with 999

missing_df.replace(to_replace=[1.0, 3.0], value=[2.0, 4.0]) # Replace several values

missing_df.replace(to_replace=[1.0, 6.5], value=np.nan) # Replace multiple values at once

## Discretization and Binning

---

Continuous data is often discretized or otherwise separated into `bins` for analysis. Suppose you have data about a group of people in a study, and you want to group them into discrete buckets.

Let discretize `weight` column.

In [ ]:
athletes.head()

In [ ]:
athletes["weight"].describe()

Before we discretize weight column, let calculate `BMI - Body Mass Index` and then discretize weight according to that values.

In [ ]:
athletes["bmi"] = athletes["weight"] / (athletes["height"] ** 2)

athletes.head()

In [ ]:
bins = [0, 18.5, 25, 30, 60]

names = ["underweight", "normal_weight", "overweight", "obese"]

athletes["new_weight"] = pd.cut(athletes["bmi"], bins=bins, labels=names)

In [ ]:
athletes.head()

In [ ]:
athletes["new_weight"].value_counts()

If we pass an integer number of bins to cut instead of explicit bin edges, Pandas `cut()` will compute equal-length bins based on the minimum and maximum values in the data.

In [ ]:
pd.cut(athletes["bmi"], 4)

Considering the distribution of the data, `cut()` method may not return equal-sized bins, while `qcut()` method by definition return approximately equal-size bins as it bins the data based on sample quantiles.

In [ ]:
pd.qcut(athletes["bmi"], q=10) # Deciles

pd.qcut(athletes["bmi"], q=4) # Quartiles

## Detecting Outliers

---

![alt text](images/iqr.png "Title")



The naive approach to detect outliers is to use `InterQuartile Range - IQR`. We can use that approach to check if the `height` column contains some outliers. The formula for `IQR` is the following:

$$
\text{IQR} = Q_{3} - Q_{1}
$$

where, $Q_{3}$ and $Q_{1}$ are upper and lower quartiles, respectively.


From the above picture, we see that $99\%$ of observations should be inside $\{Q_{1} - 1.5 \times \text{IQR}; Q_{3} + 1.5 \times \text{IQR}\}$

In [ ]:
q1 = athletes["height"].quantile(.25)
q3 = athletes["height"].quantile(.75)

iqr = q3 - q1

pmin = q1 - 1.5 * iqr
pmax = q3 + 1.5 * iqr

In [ ]:
athletes[athletes["height"].between(pmin, pmax)] # Values between IQR range

In [ ]:
athletes[(athletes["height"].lt(pmin)) | (athletes["height"].gt(pmax))] # Values outside IQR range

## String Manipulations

---

Strings represent letters and other symbols surrounded by quotation marks. Pandas has support of string manipulation and the methods are accessible by `.str` attribute. Strings are represented as `object` data type in Pandas, instead of conventional `str`.

#### Reference


[Working with text data](https://pandas.pydata.org/pandas-docs/stable/user_guide/text.html)

In [ ]:
titanic = pd.read_csv("data/titanic.csv")

titanic.head()

In [ ]:
titanic["Name"].str.split(".") # Split string on a specified character

In [ ]:
titanic["Name"].str.split(".", expand=True) # Split string on a specified character and return DataFrame

In [ ]:
titanic["Name"].str.strip() # Remove leading and trailing spaces

In [ ]:
titanic["Name"].str.contains("Mrs") # True if sub-string is included in string

In [ ]:
titanic["Name"].str.replace("Mrs", "###") # Replace string with other value

In [ ]:
titanic["Name"].str.startswith("Mrs") # True if string starts with "Mrs"

titanic["Name"].str.endswith("a") # True if string ends with "a"

In [ ]:
titanic["Name"].str.lower() # Lower case letter


titanic["Name"].str.upper() # Upper case letters

## Variable Transformation

---

To do a modeling, at the first stage, one have to take into consideration that we have some fixed set of models and we have to fit the data to our model, and the second is that these models have their assumptions - **which rarely holds in real world**.


When the assumptions do not hold, we apply different transformations to our data, in order to have as desirable data format for the model as possible. By doing so, we try to extract as much information from our data as possible.


> **The type of variable transformation greatly depends on the type of model we plan to use for modeling.**


Transformation methods are classified in two broad class:


* **Numeric Variable Transformation** - is turning a numeric variable to another numeric variable. Typically it is meant to change the scale of values and/or to adjust the skewed data distribution to Gaussian-like distribution through some `monotonic transformation`


* **Categorical Variable Transformation** - is turning a categorical variable to a numeric variable. Categorical variable transformation is mandatory for most of the machine learning models because they can handle only numeric values.

### Numerical Variable Transformations

---


* **Standardization**


* **Min-max scaling**


* **Logarithmic transformation**

In [ ]:
athletes.head()

Standardization happens using the following formula:

$$
X'_{i} = \frac{X_{i} - \bar{X_{n}}}{s}
$$

where, $\bar{X_{n}}$ is an arithmetic average and $s$ is standard deviation.

In [ ]:
(athletes['weight'] - athletes["weight"].mean()) / athletes["weight"].std()

Min-Max Scaling happens using the following formula:

$$
X'_{i} = \frac{X_{i} - min(X)}{max(X) - min(X)}
$$

In [ ]:
(athletes['weight'] - athletes["weight"].min()) / (athletes["weight"].max() - athletes["weight"].min())

Logarithmic transformation happens using by natural logarithm. However, we can use logarithm with any base. Also, note that in order to have successful logarithmic transformation the data should not contain zeros or values less than zeros.

In [ ]:
np.log(athletes["weight"]) # Natural logarithm

### Categorical Variable Transformations

---

* **One-hot encoding**

One-hot encoding is also known as dummy variable, meaning that we create indicator or binary variable containing only zeros and ones. Pandas has built in functionality for dummy variable generation. The best candidate for dummy variable is column `sex`.

In [ ]:
athletes.head()

In [ ]:
pd.get_dummies(athletes["sex"]) # Returns dummy variable for any categorical variable

In column `sex` we have two values, `female` and `male`. Have a look at dummy variables above. **They are same**. In the `female` column, 0 denotes male and 1 denotes female, while in `male` column everything is in opposite direction. We conclude that we need to drop one dummy variable, since we have duplicate values.

In [ ]:
pd.get_dummies(athletes["sex"], drop_first=True) # Drops one dummy variable

This is not an exhaustive list of variable transformations. We will cover them throughout the course.

# Summary

---

In this lecture, we saw how to deal with missing data, duplicate data, and how to do string manipulation, and some other analytical data transformations. In the next lecture, we focus on combining and rearranging datasets in various ways.